#https://pythonology.eu/a-rag-web-scraper-with-langchain-ollama-and-chroma/

In [2]:
!pip install requests
!pip install beautifulsoup4
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 59.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 65.4 MB/s eta 0:00:00


In this step, we’re using BeautifulSoup’s SoupStrainer to focus on the “content-area” class, ensuring we only extract relevant information. 

The WebBaseLoader then fetches and processes the specified URL.

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer(class_=("content-area"))
loader = WebBaseLoader(
    web_paths=("https://pythonology.eu/using-pandas_ta-to-generate-technical-indicators-and-signals/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
!pip install langchain-text-splitters

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
!docker exec -it ollama sh
#container name is assumed to be ollama
#https://github.com/JAlcocerT/Docker/tree/main/AI_Gen/Ollama

In [7]:
!ollama run llama3.2:1b

/bin/bash: line 1: ollama: command not found


In [ ]:
!ollama pull all-minilm

In [9]:
!pip install langchain-chroma

#https://pypi.org/project/langchain-chroma/


  Using cached pypika-0.48.9-py2.py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 63.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 52.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4


In [11]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

# local_embeddings = OllamaEmbeddings(model="all-minilm")
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

from langchain_community.embeddings import OllamaEmbeddings
local_embeddings = OllamaEmbeddings(model="all-minilm", base_url="http://192.168.0.12:11434")

In [13]:
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)


In [14]:
question = "what are the oversold and overbought periods?"
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke(question)

In [15]:
context = ' '.join([doc.page_content for doc in retrieved_docs])


In [16]:
!pip install -U langchain-ollama

In [19]:
# from langchain_ollama.llms import OllamaLLM

# llm = OllamaLLM(model="llama3.2:1b")

from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="llama3.2:1b", base_url="http://192.168.0.12:11434")

response = llm.invoke(f"""Answer the question according to the context given very briefly:
           Question: {question}.
           Context: {context}
""")

In [20]:
response

'There are several oversold and overbought periods:\n\n- Overbought: The Relative Strength Index (RSI) is above 70 (30-70)\n- Oversold: The Relative Strength Index (RSI) is below 30 (70-30)\n\nThese levels can be used to generate buy or sell signals based on technical analysis strategies.'

In [24]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama.llms import OllamaLLM
#from langchain.chains import RetrievalQA

def get_response_from_website_with_chunks(
    ollama_base_url: str,
    embedding_model_name: str,
    llm_model_name: str,
    question: str,
    website_url: str,
    content_class: str = "content-area",
    chunk_size: int = 1200,
    chunk_overlap: int = 100,
    search_kwargs_k: int = 3
) -> str:
    """
    Scrapes a website, creates embeddings using Ollama,
    stores them in Chroma, retrieves relevant chunks, and
    answers a question using an Ollama language model,
    following the provided code chunk structure.

    Args:
        ollama_base_url: The base URL of your Ollama server (e.g., "http://localhost:11434").
        embedding_model_name: The name of the Ollama embedding model (e.g., "all-minilm").
        llm_model_name: The name of the Ollama language model to use (e.g., "llama3.2:1b").
        question: The question you want to ask about the website content.
        website_url: The URL of the website to process.
        content_class: The CSS class name of the main content area on the website.
        chunk_size: The size of each text chunk.
        chunk_overlap: The overlap between adjacent text chunks.
        search_kwargs_k: The number of top documents to retrieve.

    Returns:
        A string containing the answer to the question based on the website content.
    """
    try:
        # Step 1: Web Scraping with LangChain
        bs4_strainer = bs4.SoupStrainer(class_=(content_class))
        loader = WebBaseLoader(
            web_paths=(website_url,),
            bs_kwargs={"parse_only": bs4_strainer},
        )
        docs = loader.load()

        # Step 2: Text Splitting for Manageable Chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap, add_start_index=True
        )
        all_splits = text_splitter.split_documents(docs)

        # Step 3: Generating Embeddings with Ollama and Chroma
        local_embeddings = OllamaEmbeddings(base_url=ollama_base_url, model=embedding_model_name)
        vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

        # Step 4: Implementing the Retrieval System
        retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": search_kwargs_k})
        retrieved_docs = retriever.invoke(question)
        context = ' '.join([doc.page_content for doc in retrieved_docs])

        # Step 5: Generating Responses with a Language Model
        llm = OllamaLLM(base_url=ollama_base_url, model=llm_model_name)
        response = llm.invoke(f"""Answer the question according to the context given very briefly:
                   Question: {question}.
                   Context: {context}
        """)
        return response

    except Exception as e:
        return f"An error occurred: {e}"

if __name__ == "__main__":
    ollama_url = "http://192.168.0.12:11434"  # Replace with your Ollama server URL if different
    embedding_model = "all-minilm"
    llm_model = "llama3.2:1b"
    user_question = "what are the oversold and overbought periods?"
    target_website = "https://pythonology.eu/using-pandas_ta-to-generate-technical-indicators-and-signals/"
    content_area_class = "content-area" # Use the same class as in the example

    answer = get_response_from_website_with_chunks(
        ollama_url,
        embedding_model,
        llm_model,
        user_question,
        target_website,
        content_class=content_area_class
    )
    print(f"Question: {user_question}")
    print(f"Answer: {answer}")

Question: what are the oversold and overbought periods?
Answer: - Oversold: Below 20 
- Overbought: Above 80


In [29]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama.llms import OllamaLLM
#from langchain.chains import RetrievalQA

def get_response_from_website_with_chunks(
    ollama_base_url: str,
    embedding_model_name: str,
    llm_model_name: str,
    question: str,
    website_url: str,
    content_class: str = "MuiBox-root css-rcazos", #class="MuiBox-root css-16nvqld" class="MuiBox-root css-rcazos"
    chunk_size: int = 1200,
    chunk_overlap: int = 100,
    search_kwargs_k: int = 3
) -> str:
    """
    Scrapes a website, creates embeddings using Ollama,
    stores them in Chroma, retrieves relevant chunks, and
    answers a question using an Ollama language model,
    following the provided code chunk structure.

    Args:
        ollama_base_url: The base URL of your Ollama server (e.g., "http://localhost:11434").
        embedding_model_name: The name of the Ollama embedding model (e.g., "all-minilm").
        llm_model_name: The name of the Ollama language model to use (e.g., "llama3.2:1b").
        question: The question you want to ask about the website content.
        website_url: The URL of the website to process.
        content_class: The CSS class name of the main content area on the website.
        chunk_size: The size of each text chunk.
        chunk_overlap: The overlap between adjacent text chunks.
        search_kwargs_k: The number of top documents to retrieve.

    Returns:
        A string containing the answer to the question based on the website content.
    """
    try:
        # Step 1: Web Scraping with LangChain
        bs4_strainer = bs4.SoupStrainer(class_=(content_class))
        loader = WebBaseLoader(
            web_paths=(website_url,),
            bs_kwargs={"parse_only": bs4_strainer},
        )
        docs = loader.load()

        # Step 2: Text Splitting for Manageable Chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, chunk_overlap=chunk_overlap, add_start_index=True
        )
        all_splits = text_splitter.split_documents(docs)

        # Step 3: Generating Embeddings with Ollama and Chroma
        local_embeddings = OllamaEmbeddings(base_url=ollama_base_url, model=embedding_model_name)
        vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

        # Step 4: Implementing the Retrieval System
        retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": search_kwargs_k})
        retrieved_docs = retriever.invoke(question)
        context = ' '.join([doc.page_content for doc in retrieved_docs])

        # Step 5: Generating Responses with a Language Model
        llm = OllamaLLM(base_url=ollama_base_url, model=llm_model_name)
        response = llm.invoke(f"""Answer the question according to the job offer context given. Reply in English and try to guess for which position it is in the Data Analytics Space. Justify your answer.
                   Question: {question}.
                   Context: {context}
        """)
        return response

    except Exception as e:
        return f"An error occurred: {e}"

if __name__ == "__main__":
    ollama_url = "http://192.168.0.12:11434"  # Replace with your Ollama server URL if different
    embedding_model = "all-minilm"
    llm_model = "llama3.2:1b"
    user_question = "what it is this offer about?"
    target_website = "https://justjoin.it/job-offer/link-group-product-manager-warszawa-ai"
    content_area_class = "MuiBox-root css-rcazos" # Use the same class as in the example

    answer = get_response_from_website_with_chunks(
        ollama_url,
        embedding_model,
        llm_model,
        user_question,
        target_website,
        content_class=content_area_class
    )
    print(f"Question: {user_question}")
    print(f"Answer: {answer}")

Question: what it is this offer about?
Answer: Based on the job offer context and the provided Zadania (duties), I would guess that this is an opportunity for a Data Analyst or Business Intelligence Analyst position in the data analytics space.

The specific tasks mentioned, such as:

* Definiowanie wizji, celów i kierunków rozwoju w oparciu o potrzeby rynku oraz cele organizacji (Tworzenie i wdrażanie strategii produktowej),
* Zarządzanie cyklem życia produktów GenAI (łączenie wiedzy technicznej z perspektywą biznesową, etc.),
* Współpraca z zespołami scrumowymi (priorytetyzacja backlogu, planowanie sprintów, etc.) oraz
* Analiza rynku i potrzeb klientów (Analiza rynku i potrzeb klientów) are all typical tasks associated with data analyst or business intelligence analyst roles.

Additionally, the mention of "KPI" (Key Performance Indicators), " danych rynkowe" (market data), and "informacje zwrotnej od klientów" (client feedback data) also suggest a role in data analysis and managemen

In [30]:
!pip freeze > requirements-export.txt